In [ ]:
!wget https://github.com/SenchaBrest/hub/releases/download/TMLCr2/DATASET_32x32.zip

In [2]:
!unzip -q DATASET_32x32.zip

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [4]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [5]:
dataset = ImageFolder(root='./DATASET_32x32', transform=transform)

In [6]:
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [7]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [ ]:
model = models.mobilenet_v2(pretrained=True)

In [3]:
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False)
model.features[0][1] = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [4]:
num_classes = 29
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(correct, total)
print(f"Validation Accuracy: {accuracy * 100}%")

In [ ]:
torch.save(model.state_dict(), 'model_32x32.pth')
from google.colab import files
files.download('model_32x32.pth')

# To ONNX

In [ ]:
!pip install onnx

In [8]:
import torch.onnx

device = torch.device("cpu")
model.to(device)

dummy_input = torch.randn(1, 1, 32, 32)
torch.onnx.export(model, dummy_input, 'model32.onnx', verbose=True, input_names=['input'], output_names=['output'])

In [10]:
from google.colab import files
files.download('model32.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>